In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import requests
import bs4 as bs
import urllib.request

In [2]:
filename = 'nlp_model2.pkl'
clf = pickle.load(open(filename, 'rb'))
vectorizer = pickle.load(open('tranform.pkl','rb'))

In [3]:
def rcmd(m):
    m = m.lower()
    data, similarity = create_similarity()
    if m not in data['movie_title'].unique():
        return("Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies")
    else:
        i = data.loc[data['movie_title']==m].index[0]
        lst = list(enumerate(similarity[i]))
        lst = sorted(lst, key = lambda x:x[1] ,reverse=True)
        lst = lst[1:11]
        l = []
        for i in range(len(lst)):
            a = lst[i][0]
            l.append(data['movie_title'][a])
        return l

In [4]:
def create_similarity():
    data = pd.read_csv('final_data.csv')
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(data['comb'])
    similarity = cosine_similarity(count_matrix)
    return data,similarity

In [5]:
def reviews (m):
    imdb_id=get_imdb_id(m)
    sauce = urllib.request.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read()
    soup = bs.BeautifulSoup(sauce,'lxml')
    soup_result = soup.find_all("div",{"class":"text show-more__control"})
    if(len(soup_result)>0):
        return (soup_result)
    else:
        return ("Reviews Not Accessible! Kindly Search for Another Movie!!")

In [6]:
def get_imdb_id (m):
    m = m.lower()
    api_key="f107e39c1038a32e3f1fec8a62383ec1"
    r = requests.get(url = "https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&query="+m).json()
    if(len(r["results"])>0):
        movie_id=r["results"][0]["id"]
        s= requests.get(url="https://api.themoviedb.org/3/movie/{}?api_key=f107e39c1038a32e3f1fec8a62383ec1".format(movie_id)).json()
        if(len(s["imdb_id"])>0):
            return (s["imdb_id"])
        else:
            return ("Imdb ID Not Available")
    else:
        return("Movie ID not Available")

In [7]:
def sentiments(s):
    reviews_list = [] 
    reviews_status = [] 
    for reviews in s:
        if reviews.string:
            reviews_list.append(reviews.string)
            movie_review_list = np.array([reviews.string])
            movie_vector = vectorizer.transform(movie_review_list)
            pred = clf.predict(movie_vector)
            reviews_status.append('Positive' if pred else 'Negative')
    movie_reviews = {reviews_list[i]: reviews_status[i] for i in range(len(reviews_list))}  
    return(movie_reviews)

In [8]:
def similarity():
    movie = input("Enter Movie Name:")
    rc = rcmd(movie)
    print("Top 10 Movie Recommndations -- >")
    for i in rc:
          print(i)
    sc= reviews(movie)
    if(sc=="Reviews Not Accessible! Kindly Search for Another Movie!!"):
        print(sc)
    else:
        print("\n\n")
        print("Sentiment Analysis -->")
        sentiment_analysis=sentiments(sc)
        for i in sentiment_analysis:
            print(i)
            print("\t")
            print(sentiment_analysis[i])
            print("\n")

In [9]:
similarity()

Enter Movie Name:banshee chapter
Top 10 Movie Recommndations -- >
eden lake
final destination 2
saw
sleepy hollow            
haywire
red state
creepshow 2
the lords of salem
below zero
alien: covenant



Sentiment Analysis -->
I really wanted to like this film. But there was so many things being thrown in the mix it's was just too much to take. Number stations, mk ultra, drugs, government conspiracies, and a barrage of scenes which never really give the viewer time to build up to a feeling of dread. They're just delivered to you in a quick fire fashion and even the "face at the window" shots are so quick you can't define them sometimes. I have to say I liked the Ted Levine character Blackburn as he played it with gusto right to the end. But apart from the outlandish story there are many plot holes and the ending just wasn't that satisfying for me. Still, I respect any new director trying something a little different.
	
Positive


The Banshee Chapter is a decent horror flick with a lot